# Analyzing results
Question: In combined queries, does correct/incorrect prediction of earlier nutrient(s) affect accuracy of later nutrient(s)?

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [21]:
combined_df = pd.read_json("/data/lucasjia/projects/nutri/results/multi-nutrient/sub1/samples_combined_base_20250719_194516.jsonl", lines=True)

# print(combined_df.columns)
combined_results = combined_df[["doc_id", "doc", "result"]]
# print(results["result"].iloc[0])

for nutrient in ["carb", "protein", "fat", "energy"]:
    acc = []
    mae = []
    mse = []
    for result in combined_results["result"]:
        acc.append(result[nutrient]["acc"])
        mae.append(result[nutrient]["mae"])
        mse.append(result[nutrient]["mse"])

    combined_results[f"{nutrient}_acc"] = acc
    combined_results[f"{nutrient}_mae"] = mae
    combined_results[f"{nutrient }_mse"] = mse

print(combined_results.columns)


Index(['doc_id', 'doc', 'result', 'carb_acc', 'carb_mae', 'carb_mse',
       'protein_acc', 'protein_mae', 'protein_mse', 'fat_acc', 'fat_mae',
       'fat_mse', 'energy_acc', 'energy_mae', 'energy_mse'],
      dtype='object')


/tmp/ipykernel_214324/1141090224.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_results[f"{nutrient}_acc"] = acc
/tmp/ipykernel_214324/1141090224.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_results[f"{nutrient}_mae"] = mae
/tmp/ipykernel_214324/1141090224.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [22]:
files = {
    "carb": "/data/lucasjia/projects/nutri/results/multi-nutrient/sub1/samples_carb_base_20250720_033502.jsonl",
    "protein": "/data/lucasjia/projects/nutri/results/multi-nutrient/sub1/samples_protein_base_20250719_192710.jsonl",
    "fat": "/data/lucasjia/projects/nutri/results/multi-nutrient/sub1/samples_fat_base_20250719_184222.jsonl",
    "energy": "/data/lucasjia/projects/nutri/results/multi-nutrient/sub1/samples_energy_base_20250721_222318.jsonl",
}

# Init empty results dictionary
indiv_results = {}

for nutrient, filepath in files.items():
    df = pd.read_json(filepath, lines=True)
    indiv_results[f"{nutrient}_acc"] = [r["acc"] for r in df["result"]]
    indiv_results[f"{nutrient}_mae"] = [r["mae"] for r in df["result"]]
    indiv_results[f"{nutrient}_mse"] = [r["mse"] for r in df["result"]]

# Convert to final DataFrame
indiv_results_df = pd.DataFrame(indiv_results)
print(indiv_results_df.head())

   carb_acc  carb_mae  carb_mse  protein_acc  protein_mae  protein_mse  \
0      True      0.00    0.0000         True         0.00       0.0000   
1      True      5.01   25.1001        False         5.46      29.8116   
2      True      5.62   31.5844         True         0.08       0.0064   
3     False     10.11  102.2121        False         5.28      27.8784   
4      True      3.67   13.4689         True         0.98       0.9604   

   fat_acc  fat_mae  fat_mse  energy_acc  energy_mae  energy_mse  
0     True     0.00   0.0000        True        0.00      0.0000  
1    False     5.68  32.2624        True       15.96    254.7216  
2    False     3.08   9.4864        True        1.75      3.0625  
3    False     2.60   6.7600        True       37.25   1387.5625  
4    False     4.04  16.3216       False       65.28   4261.4784  


In [ ]:
# correlation between mae for combined estimates vs individual estimates
combined_mae = combined_results[["carb_mae", "protein_mae", "fat_mae", "energy_mae"]]
indiv_mae = indiv_results_df[["carb_mae", "protein_mae", "fat_mae", "energy_mae"]]
print(combined_mae.corr())
print(indiv_mae.corr())

             carb_mae  protein_mae   fat_mae  energy_mae
carb_mae     1.000000     0.507155  0.273352    0.621870
protein_mae  0.507155     1.000000  0.507924    0.631438
fat_mae      0.273352     0.507924  1.000000    0.788847
energy_mae   0.621870     0.631438  0.788847    1.000000
             carb_mae  protein_mae   fat_mae  energy_mae
carb_mae     1.000000     0.446933  0.212044    0.515180
protein_mae  0.446933     1.000000  0.270500    0.515830
fat_mae      0.212044     0.270500  1.000000    0.585662
energy_mae   0.515180     0.515830  0.585662    1.000000


In [24]:
def match_percentage_matrix(df):
    cols = df.columns
    n = len(df)
    match_matrix = pd.DataFrame(index=cols, columns=cols, dtype=float)

    for col1 in cols:
        for col2 in cols:
            match_matrix.loc[col1, col2] = np.mean(df[col1] == df[col2]) * 100  # percentage match

    return match_matrix

In [ ]:
# same with acc
combined_acc = combined_results[["carb_acc", "protein_acc", "fat_acc", "energy_acc"]]
indiv_acc = indiv_results_df[["carb_acc", "protein_acc", "fat_acc", "energy_acc"]]

acc_match_combined = match_percentage_matrix(combined_acc)
acc_match_indiv = match_percentage_matrix(indiv_acc)

print("Match % (Combined Acc):\n", acc_match_combined)
print("Match % (Individual Acc):\n", acc_match_indiv)

Match % (Combined Acc):
                carb_acc  protein_acc     fat_acc  energy_acc
carb_acc     100.000000    69.433962   63.962264   78.113208
protein_acc   69.433962   100.000000   69.245283   72.452830
fat_acc       63.962264    69.245283  100.000000   73.018868
energy_acc    78.113208    72.452830   73.018868  100.000000
Match % (Individual Acc):
                carb_acc  protein_acc     fat_acc  energy_acc
carb_acc     100.000000    68.867925   65.849057   76.603774
protein_acc   68.867925   100.000000   68.679245   69.245283
fat_acc       65.849057    68.679245  100.000000   69.622642
energy_acc    76.603774    69.245283   69.622642  100.000000
